In [71]:
import torch
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse import csr_matrix

import torch
import networkx as nx
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
def generate_dense_connected_graph(n, density, max_degree):
    """
    生成一个 n 阶稠密连通简单图的邻接矩阵，且每个节点的最大度数不超过 max_degree。

    参数:
        n (int): 图的节点数。
        density (float): 图的密度，取值范围为 (0, 1]，默认值为 0.5。
        max_degree (int): 每个节点的最大度数，默认值为 20。

    返回:
        torch.Tensor: 生成的邻接矩阵，类型为 float32。
    """
    # 初始化邻接矩阵
    graph = torch.zeros((n, n), dtype=torch.float32)

    # 确保图至少有一条边
    for i in range(n - 1):
        graph[i][i + 1] = 1.0
        graph[i + 1][i] = 1.0

    # 生成最小生成树（确保连通性）
    while True:
        sparse_graph = csr_matrix(graph.numpy())
        mst = minimum_spanning_tree(sparse_graph)
        mst = torch.tensor(mst.toarray(), dtype=torch.float32)
        connected_graph = (mst + mst.T) > 0
        connected_graph = connected_graph.float().to(torch.float32)

        # 检查是否连通
        G = nx.Graph(connected_graph.numpy())
        if nx.is_connected(G):
            break

    # 随机添加一些边以增加稠密度，同时控制最大度数
    for i in range(n):
        for j in range(i + 1, n):
            if torch.rand(1, dtype=torch.float32).item() < density and \
               G.degree[i] < max_degree and G.degree[j] < max_degree:
                connected_graph[i][j] = 1.0
                connected_graph[j][i] = 1.0
                G.add_edge(i, j)

    return connected_graph

A = generate_dense_connected_graph(10000, 0.002, 2000)

In [72]:
import torch

def lra_2_hop(A):
    P2 = A @ A
    P2.fill_diagonal_(0)
    return P2

def lra_3_hop(A):
    degrees = A.sum(dim=1).unsqueeze(0)
    P2 = lra_2_hop(A)
    Q3 = P2 @ A
    P3 = Q3 - A * (degrees - 1)
    P3.fill_diagonal_(0)
    return P3

def lra_4_hop(A):
    P3 = lra_3_hop(A)
    P2 = lra_2_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    degrees = A.sum(dim=1).unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)

    P4_0 = Q4 - (degrees - 1) * P2
    P4_1 = Q4 - (degrees - 2) * P2 - (Q3_diag - 2 * P2)

    P4 = P4_0 * (1 - A) + P4_1 * A
    P4.fill_diagonal_(0)
    return P4

def lra_5_hop(A):
    P2 = lra_2_hop(A)
    P3 = lra_3_hop(A)
    P4 = lra_4_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    Q5 = P4 @ A

    degrees = A.sum(dim=1).unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)
    Q4_diag = Q4.diagonal().unsqueeze(0)

    P5_0 = Q5.clone()
    P5_0 -= (degrees - 1) * P3 - A @ (A * P2)
    P5_0 -= (A @ A) * Q3_diag - 2 * A @ (A * P2)

    P5_1 = Q5.clone()
    P5_1 -= (degrees - 1) + (degrees - 2) * P3 - A @ (A * P2)
    P5_1 -= (A @ A) * (Q3_diag - 2 * P2 + 2) - 2 * A @ (A * P2)
    P5_1 -= Q4_diag - 2 * P3 - P2 * (P2 - 1)

    P5 = P5_0 * (1 - A) + P5_1 * A
    P5.fill_diagonal_(0)
    return P5

def lra_k_hop(A, k):
    if k == 1:
        return A
    elif k == 2:
        return lra_2_hop(A)
    elif k == 3:
        return lra_3_hop(A)
    elif k == 4:
        return lra_4_hop(A)
    elif k == 5:
        return lra_5_hop(A)

    P2 = lra_2_hop(A)
    P3 = lra_3_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    Q4_diag = Q4.diagonal().unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)

    degrees = A.sum(dim=1).unsqueeze(0)

    Pk = lra_k_hop(A, k - 1) @ A
    Pk -= (degrees - 1) + (degrees - 2) * P3 - A @ (A * P2)
    Pk -= (A @ A) * (Q3_diag - 2 * P2 + 2) - 2 * A @ (A * P2)
    Pk -= Q4_diag - 2 * P3 - P2 * (P2 - 1)
    Pk.fill_diagonal_(0)
    return Pk

def generate_q_matrices(A):
    A = A.to(torch.float64)  # 统一转换为float64类型
    n = A.size(0)
    Q_matrices = []
    I = torch.eye(A.size(0), dtype=torch.float64)  # 使用浮点类型
    Q_matrices.append(I)

    for k in range(1, 8):
        Qk = lra_k_hop(A, k)
        Q_matrices.append(Qk)

    return Q_matrices

In [73]:
lra_5_hop(A)

tensor([[  0., 183., 498.,  ..., 604., 481., 397.],
        [168.,   0., 150.,  ..., 259., 241., 163.],
        [498., 161.,   0.,  ..., 503., 463., 355.],
        ...,
        [604., 259., 503.,  ...,   0., 488., 383.],
        [481., 241., 463.,  ..., 486.,   0., 302.],
        [397., 163., 355.,  ..., 383., 296.,   0.]])

In [74]:
lra_k_hop(A,6)

tensor([[    0.,  4639., 10873.,  ..., 12911., 12470.,  8638.],
        [ 5179.,     0.,  4376.,  ...,  5364.,  4989.,  3735.],
        [10942.,  4008.,     0.,  ..., 10998., 11176.,  7543.],
        ...,
        [12893.,  5135., 10918.,  ...,     0., 11479.,  8398.],
        [12498.,  4839., 11120.,  ..., 11532.,     0.,  8043.],
        [ 8776.,  3691.,  7578.,  ...,  8486.,  8367.,     0.]])

In [75]:
lra_k_hop(A,7)

tensor([[     0., 101685., 254208.,  ..., 291810., 281276., 200226.],
        [114268.,      0.,  99999.,  ..., 118573., 116352.,  81012.],
        [255761.,  90952.,      0.,  ..., 256364., 249189., 176509.],
        ...,
        [292159., 115624., 254874.,  ...,      0., 268912., 197898.],
        [281632., 113573., 247862.,  ..., 269212.,      0., 181042.],
        [202862.,  80040., 178298.,  ..., 200871., 189677.,      0.]])

In [76]:
'''
The corrected code correctly calculates the number of loop - free paths of order 5.
'''


import torch
def lra_5_hop_revised(A):
    P2 = lra_2_hop(A)
    P3 = lra_3_hop(A)
    P4 = lra_4_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    Q5 = P4 @ A

    degrees = A.sum(dim=1).unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)
    Q4_diag = Q4.diagonal().unsqueeze(0)

    P5_0 = Q5.clone()
    P5_0 -= (degrees - 1) * P3 - A @ (A * P2)
    P5_0 -= (A @ A) * Q3_diag - 2 * A @ (A * P2)

    P5_1 = Q5.clone()
    P5_1 -= (degrees - 2) * P3 - A @ (A * (P2-1))
    P5_1 -= (A @ A) * (Q3_diag - 2 * P2 + 2) - 2 * A @ (A * P2)
    P5_1 -= Q4_diag - 2 * P3 - P2 * (P2 - 1)

    P5 = P5_0 * (1 - A) + P5_1 * A
    P5.fill_diagonal_(0)
    return P5

def lra_k_hop_revised(A, k):
    if k == 1:
        return A
    elif k == 2:
        return lra_2_hop(A)
    elif k == 3:
        return lra_3_hop(A)
    elif k == 4:
        return lra_4_hop(A)
    elif k == 5:
        return lra_5_hop_revised(A)

    P2 = lra_2_hop(A)
    P3 = lra_3_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    Q4_diag = Q4.diagonal().unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)

    degrees = A.sum(dim=1).unsqueeze(0)

    Pk = lra_k_hop(A, k - 1) @ A
    Pk -= (degrees - 2) * P3 - A @ (A * (P2-1))
    Pk -= (A @ A) * (Q3_diag - 2 * P2 + 2) - 2 * A @ (A * P2)
    Pk -= Q4_diag - 2 * P3 - P2 * (P2 - 1)
    Pk.fill_diagonal_(0)
    return Pk

def generate_q_matrices_revised(A):
    A = A.to(torch.float64)  # 统一转换为float64类型
    n = A.size(0)
    Q_matrices = []
    I = torch.eye(A.size(0), dtype=torch.float64)  # 使用浮点类型
    Q_matrices.append(I)

    for k in range(1, 8):
        Qk = lra_k_hop_revised(A, k)
        Q_matrices.append(Qk)

    return Q_matrices

In [77]:
lra_5_hop_revised(A)

tensor([[  0., 191., 498.,  ..., 604., 481., 397.],
        [191.,   0., 169.,  ..., 259., 241., 163.],
        [498., 169.,   0.,  ..., 503., 463., 355.],
        ...,
        [604., 259., 503.,  ...,   0., 509., 383.],
        [481., 241., 463.,  ..., 509.,   0., 317.],
        [397., 163., 355.,  ..., 383., 317.,   0.]])

In [78]:
lra_k_hop_revised(A, 6)

tensor([[    0.,  4647., 10891.,  ..., 12934., 12491.,  8653.],
        [ 5202.,     0.,  4395.,  ...,  5387.,  5010.,  3750.],
        [10964.,  4016.,     0.,  ..., 11021., 11197.,  7557.],
        ...,
        [12916.,  5143., 10937.,  ...,     0., 11500.,  8411.],
        [12521.,  4847., 11139.,  ..., 11555.,     0.,  8058.],
        [ 8799.,  3699.,  7596.,  ...,  8507.,  8388.,     0.]])

In [79]:
lra_k_hop_revised(A, 7)

tensor([[     0., 101693., 254226.,  ..., 291833., 281297., 200241.],
        [114291.,      0., 100018.,  ..., 118596., 116373.,  81027.],
        [255783.,  90960.,      0.,  ..., 256387., 249210., 176523.],
        ...,
        [292182., 115632., 254893.,  ...,      0., 268933., 197911.],
        [281655., 113581., 247881.,  ..., 269235.,      0., 181057.],
        [202885.,  80048., 178316.,  ..., 200892., 189698.,      0.]])

In [80]:
import torch
def diag(matrix):
    """
    Input: An n×n PyTorch tensor (square matrix)
    Output: An n×n diagonal matrix with the same diagonal elements
    """
    # Extract the diagonal elements
    diag_elements = matrix.diagonal()
    
    # Create diagonal matrix using torch.diag()
    diag_matrix = torch.diag(diag_elements)
    
    return diag_matrix

In [81]:
def common_neighbor_similarity_sparse_cpu(A):
    dense_common = A@A
    valid_pairs = torch.nonzero(dense_common, as_tuple=False)
    
    num_vertices = A.shape[0]
    similarity = torch.zeros(num_vertices, num_vertices, dtype=torch.float32)
    
    # 批处理所有有效顶点对
    for i, j in valid_pairs:
        i, j = i.item(), j.item()
        num_common = int(dense_common[i, j].item())
        
        if num_common < 2:
            continue
            
        # 获取公共邻居索引
        common_neighbor_indices = (A[i].to_dense() * A[j].to_dense()).nonzero(as_tuple=True)[0]
        
        # 使用矩阵运算计算相邻对数
        neighbors_adj = A[common_neighbor_indices][:, common_neighbor_indices].to_dense()
        count_adjacent = neighbors_adj.sum().item()
        
        total_pairs = num_common * num_common
        similarity[i, j] = count_adjacent / total_pairs
    
    return similarity

B = common_neighbor_similarity_sparse_cpu(A)
print(B)

tensor([[0.0035, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0069, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0041, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [82]:
import torch

def diag_matmul_left(D_diag, M):
    """
    高效计算 对角矩阵D × 普通矩阵M
    
    参数:
    D_diag (torch.Tensor): 对角矩阵D的对角元素，形状为 [n]
    M (torch.Tensor): 普通矩阵，形状为 [n, m]
    
    返回:
    torch.Tensor: 结果矩阵，形状为 [n, m]
    """
    diagonal = torch.diag(D_diag)           # 提取对角线元素 (n,)
    column_vector = diagonal.unsqueeze(1)   # 转换为列向量 (n, 1)
    return column_vector * M  # 等价于 D @ M，但更高效

def diag_matmul_right(M, D_diag):
    """
    高效计算 普通矩阵M × 对角矩阵D
    
    参数:
    M (torch.Tensor): 普通矩阵，形状为 [m, n]
    D_diag (torch.Tensor): 对角矩阵D的对角元素，形状为 [n]
    
    返回:
    torch.Tensor: 结果矩阵，形状为 [m, n]
    """
    return M * torch.diag(D_diag)  # 等价于 M @ D，但更高效

In [83]:
import torch

A2 = torch.matrix_power(A, 2)
A3 = torch.matrix_power(A, 3)
A4 = torch.matrix_power(A, 4)
A5 = torch.matrix_power(A, 5)
A6 = torch.matrix_power(A, 6)
D2 = diag(A2)
D3 = diag(A3)
D4 = diag(A4)
D5 = diag(A5)
D6 = diag(A6)
AD2A = A@diag_matmul_left(D2,A)
D22 = D2*D2
A2D2 = diag_matmul_right(A2,D2)
D3A = diag_matmul_left(D3,A)
AD3 = diag_matmul_right(A,D3)


# 优化后的表达式
P6 = (A6 - (
    diag_matmul_left(D2,A4) + diag_matmul_right(A4,D2)
    + diag_matmul_right(A,D2)@A3 + diag_matmul_right(A3,D2)@A
    + A2D2@A2 + AD3@A2
    + A2@D3A + diag_matmul_right(A,D4)@A + diag_matmul_left(D3,A3)
    + diag_matmul_right(A3,D3) + diag_matmul_right(A2,D4) + D4@A2
    + diag_matmul_right(A,D5) + diag_matmul_left(D5,A) + D6
)+(
    4*A4 + diag_matmul_right(A2,diag(AD2A)) + diag_matmul_left(diag(AD2A),A2)
    + diag_matmul_left(D2,A2D2) + 2*diag_matmul_right(A,D22)@A
    + 2*diag_matmul_left(D22,A2) + 2*diag_matmul_right(A2,D22)
    + A@diag_matmul_left(diag(AD2A),A) + diag_matmul_right(AD2A,D2) + diag_matmul_left(D2,AD2A)
    + 4*diag_matmul_right(A,(D2*D3)) + 4*diag_matmul_left(D2,D3A) + 3*A@(A2*A)@A
    + 2*diag_matmul_right(A,diag(A2D2@A)) + 2*diag_matmul_left(diag(A2D2@A),A) + 3*A2@(A2*A)
    + 3*(A*A2)@A2 + diag_matmul_right(A,diag(A@D3A)) + diag_matmul_left(diag(A@D3A),A) + diag_matmul_left(D2,AD3)
    + diag_matmul_right(D3A,D2) + 3*A@(A*A3) + 3*(A*A3)@A + diag(A@D4@A)
    + 4*D2*D4 + 2*diag(A@diag_matmul_left(D2,A3)) + diag(A2D2@A2)
    + 3*A*A4 + 3*A@(A*A2*A2) + 3*(A*A2*A2)@A
    + 2*diag(AD3@A2) + 2*(D3*D3) + 6*A*A2*A3
    + A2*A2*A2
)-(
    10*AD2A + 2*diag(A@diag_matmul_left(D22,A)) + 6*D22*D2
    + 7*diag_matmul_left(D2,A2) + 7*A2D2 + diag(A@diag_matmul_left(diag(AD2A),A))
    + 4*D2*diag(AD2A) + 13*AD3 + 13*D3A
    + 3*A*(AD2A) + 10*D4 + 4*A2*A2 + 4*A@(A*A2)
    + 4*(A*A2)@A + 12*diag_matmul_right((A*A2),D2) + 12*diag_matmul_left(D2,(A*A2))
    + 12*diag(torch.matrix_power((A*A2), 2)) + 3*diag(A@(A*A2)@A) + 8*A*(A@(A*A2))
    + 8*A*((A*A2)@A)
)+(
    8*A2 + 2*diag(AD2A) + 18*D22 + 8*diag(AD2A)
    + 44*A*A2 + 4*A2 + 8*D3
)- 12*D2)

print(P6)

tensor([[    0.,  4490., 10561.,  ..., 12251., 12074.,  8295.],
        [ 4490.,     0.,  3889.,  ...,  5003.,  4731.,  3644.],
        [10561.,  3889.,     0.,  ..., 10584., 10577.,  7396.],
        ...,
        [12251.,  5003., 10584.,  ...,     0., 10662.,  8297.],
        [12074.,  4731., 10577.,  ..., 10662.,     0.,  7588.],
        [ 8295.,  3644.,  7396.,  ...,  8297.,  7588.,     0.]])


In [84]:
A2 = torch.matrix_power(A, 2)
A3 = torch.matrix_power(A, 3)
A4 = torch.matrix_power(A, 4)
A5 = torch.matrix_power(A, 5)
A6 = torch.matrix_power(A, 6)
A7 = torch.matrix_power(A, 7)

D2 = diag(A2)
D3 = diag(A3)
D4 = diag(A4)
D5 = diag(A5)
D6 = diag(A6)
D7 = diag(A7)
D22 = D2*D2
A2D2 = diag_matmul_right(A2,D2)
D3A = diag_matmul_left(D3,A)
AD3 = diag_matmul_right(A,D3)
D2A  = diag_matmul_left(D2,A)
AD2A = A@D2A
D2A2 = diag_matmul_left(D2,A2)
AD2 = diag_matmul_right(A,D2)
A2D2A = A2D2@A
dAD2Ad_A = diag_matmul_left(diag(AD2A),A)
AD3A = AD3@A
D3A2 = diag_matmul_left(D3,A2)
D4A = diag_matmul_left(D4,A)
AD4 = diag_matmul_right(A,D4)
bA_A2bA = (A*A2)@A
AD2A2 = AD2@A2
AbA_A2b = A@(A*A2)
D2D2A = diag_matmul_left((D2*D2),A)

P7 = (36*A) - (
    (66*AD2) + (66*D2A) + (144*D3) + (112*A2*A)
) + (
    (21*A3) + (18*diag_matmul_left(D2*D2,A)) + (18*diag_matmul_right(A,D2*D2)) 
    + (14*dAD2Ad_A) + (14*A@diag(AD2A)) + (56*bA_A2bA) + (56*AbA_A2b) 
    + (18*(D2*D3)) + (8*diag(AD3A)) + (12*diag(A2D2A)) + (47*A*A3) + (4*diag_matmul_right(D2A,D2))
    + (18*D2D2A) + (18*diag_matmul_right(A,(D2*D2))) 
    + (76*(D2*D3)) + (32*diag(AD2A2)) + (9*diag(AD3A)) + (8*diag(A2D2A)) 
    + (16*D2A@D2) + (88*A2*A*A2) + (16*A*(AD2A)) + (8*D3A) + (8*AD3) 
    + (36*A*(bA_A2bA)) + (36*A*(AbA_A2b)) + (16*diag_matmul_left(D2,(A2*A))) + (16*diag_matmul_right((A2*A),D2)) 
    + (20*diag(A@bA_A2bA)) + (32*diag((A2*A)@A2)) + (5*(A*A3)) 
    + (11*A*A2*A2*B) + (11*A2*A2*B)
) - (
    (11*AD2A2) + (11*A2D2A) + (8*D2@A3) + (8*A3@D2) + (17*AD3A) + (14*D3A2) 
    + (14*A2@D3) + (14*D4A) + (14*AD4) + (11*D5) 
    + (6*diag_matmul_left((D2*D2*D2),A)) + (6*diag_matmul_right(A,(D2*D2*D2))) 
    + (2*diag(A@D2D2A)@A) + (2*A@diag(A@D2D2A)) 
    + (12*(A*A2)@D2A) + (12*AD2@(A*A2)) + (18*(D2*D2)*D3) + (4*diag(AD3@D2A)) 
    + (4*diag(AD2@D2A2)) + (12*diag_matmul_left(D2,bA_A2bA)) + (12*diag_matmul_right(AbA_A2b,D2)) 
    + (4*diag_matmul_left(D2,dAD2Ad_A)) + (4*AD2@diag(AD2A)) + (12*diag_matmul_left(D2,(A3*A))) + (12*diag_matmul_right((A3*A),D2)) 
    + (2*diag_matmul_left((D2*D2),AD2)) + (2*diag_matmul_right(D2A,(D2*D2))) 
    + (4*A@bA_A2bA) + (4*(A*A2)@A2) + (4*A2@(A*A2)) + (diag(A@dAD2Ad_A)@A) 
    + diag_matmul_right(A,diag(A@dAD2Ad_A)) + (4*(A2*A2)@A) + (4*A@(A2*A2)) 
    + (4*D2*diag(AD3A)) + (diag(A@diag(AD3A)@A)) + (4*diag(AD2A)*D3) 
    + (2*diag(A@diag_matmul_left(diag(AD2A),A2))) + (4*A3*A2) + (diag_matmul_right(D2A,diag(AD2A))) + (diag_matmul_left(diag(AD2A),AD2)) 
    + (12*diag_matmul_left(diag(torch.matrix_power(A2*A,2)),A)) + (12*diag_matmul_right(A,diag(torch.matrix_power(A2*A,2)))) 
    + (12*diag_matmul_left(D3,(A2*A))) + (12*diag_matmul_right((A2*A),D3)) + (12*diag_matmul_left(D2,(A2*A*A2))) 
    + (12*diag_matmul_right((A2*A*A2),D2)) + (3*diag_matmul_left(diag(A@bA_A2bA),A)) + (3*A@diag(A@bA_A2bA)) + (8*(A*(bA_A2bA))@A) 
    + (8*A@(A*(AbA_A2b))) + (3*diag(A@(A*A3)@A)) 
    + (24*diag((A*A2)@A3)) + (6*diag(A2@(A2*A)@A)) + (8*(A*(AbA_A2b))@A) + (8*A@(A*(bA_A2bA))) 
    + (3*((AD2A)*A)@A) + (3*A@((AD2A)*A)) 
    + (8*A*(A@(A*A3))) + (8*A*((A*A3)@A)) + (3*(A*A2)@AD2) + (3*D2A@(A*A2)) + (8*A2*(AbA_A2b)) 
    + (8*A2*(bA_A2bA)) + (4*A*(A@(A2*A2))) + (4*A*((A2*A2)@A)) 
    + (8*D2*diag(AD2A2)) + (2*diag(A@diag_matmul_left(diag(AD2A2),A))) + (diag(AD2A@D2A)) + (6*A*A2*(AD2A)) 
    + (3*A*(AD2A2)) + (3*A*(A2D2A)) + (3*A*(AD3A)) 
    + (8*diag(A2@(A*A2*A2))) + (3*diag(A@(A*A2*A2)@A)) + (16*A*((A*A2)@(A*A2)))
) + (
    (5*A5) + (2*A@diag_matmul_left((D2*D2),A2)) + (2*A2@D2D2A) 
    + (2*diag_matmul_left((D2*D2),A3)) + (2*diag_matmul_right(A3,(D2*D2))) 
    + (diag_matmul_left(diag(AD2A),A3)) + (A3@diag(AD2A)) + (A@diag(AD2A)@A2) + (A2@dAD2Ad_A) 
    + (4*AD2@D3A) + (4*diag_matmul_left(D2,D3A2)) + (4*diag_matmul_right(A2,(D2*D3))) + (4*diag_matmul_left(D2,D4A)) 
    + (4*diag_matmul_right(AD2,D4)) + (3*AbA_A2b@A2) + (3*A2@bA_A2bA) + (3*(A*A2)@A3) + (3*A3@(A*A2)) 
    + (A@diag(AD3A)@A) + (3*A@(A*A3)@A) + (diag_matmul_left(diag(AD3A),A2)) 
    + (A2@diag(AD3A)) + (3*(A*A3)@A2) + (3*A2@(A*A3)) + (diag_matmul_left(diag(AD4@A),A)) + (A@diag(AD4@A)) 
    + (3*(A*A4)@A) + (3*A@(A*A4)) + (2*A@diag(A2D2A)@A) 
    + (2*diag_matmul_left(diag(AD2A2),A2)) + (2*A2@diag(AD2A2)) + (AD2A@D2A) + (D2A@D2A2) + (diag_matmul_right(A2D2A,D2)) 
    + (2*diag(AD2@A3)@A) + (2*A@diag(AD2@A3)) + (diag_matmul_left(D3,AD2A)) 
    + (diag_matmul_right(AD2A,D3)) + (diag_matmul_left(diag(A2@D2A2),A)) + (A@diag(A2@D2A2)) + (4*(D2*D5)) + (diag(diag_matmul_right(A,D5)@A)) 
    + (2*diag(A4@D2A)) + (2*diag(A3@D2A2)) + (diag_matmul_right(AD2A2,D2)) + (D2A2@D2A) 
    + (3*A*A5) + (D2A@D3A) + (AD3@AD2) + (diag_matmul_right(D2A,D4)) + (diag_matmul_left(D4,AD2)) + (diag_matmul_right(D2A2,D3)) + (diag_matmul_right(D3A2,D2)) 
    + (diag_matmul_left(D2,A3@D2)) + (2*diag_matmul_left((D3*D3),A)) 
    + (2*diag_matmul_right(A,(D3*D3))) + (3*A@(A2*A*A2)@A) + (3*(A2*A*A2)@A2) + (3*A2@(A2*A*A2)) 
    + (2*diag_matmul_left(diag(AD3@A2),A)) + (2*A@diag(AD3@A2)) + (6*(A3*A*A2)@A) 
    + (6*A@(A3*A*A2)) + (4*(D3*D4)) + (2*diag(A2@D4A)) + (2*diag(AD3@A3)) + (diag(A2@D3A2)) 
    + (A@(A2*A2*A2)) + ((A2*A2*A2)@A) + (6*(A*A2*A4)) + diag_matmul_left(D3,AD3) 
    + (3*(A*A3*A3)) + (3*(A2*A2*A3))
) - (
    (A3@D2A2) + (A2@diag_matmul_left(D2,A3)) + (AD2@A4) + (A4@D2A) + (diag_matmul_left(D2,A5)) + (diag_matmul_right(A5,D2)) + (A2@D3A2) + (AD3@A3) 
    + (A3@D3A) + (AD4@A2) + (A2@D4A) + (diag_matmul_right(A,D5)@A) + (diag_matmul_left(D3,A4)) 
    + (diag_matmul_right(A4,D3)) + (diag_matmul_left(D4,A3)) + (diag_matmul_right(A3,D4)) + (diag_matmul_left(D5,A2)) + (diag_matmul_right(A2,D5)) 
    + (diag_matmul_left(D6,A)) + (diag_matmul_right(A,D6)) + (D7)) + A7

print(P7)

tensor([[     0.,  94951., 234500.,  ..., 266526., 260055., 186490.],
        [ 94951.,      0.,  84556.,  ..., 108434., 106929.,  75464.],
        [234500.,  84556.,      0.,  ..., 234879., 229677., 162688.],
        ...,
        [266526., 108434., 234879.,  ...,      0., 236306., 182672.],
        [260055., 106929., 229677.,  ..., 236306.,      0., 165405.],
        [186490.,  75464., 162688.,  ..., 182672., 165405.,      0.]])
